# Standard adversarial training 

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '4,5,6,7'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/standard/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.1
momentum = 0.9
batch_size = 128
total_epochs = 100
epsilon = 8/255
alpha = 2/255
num_repeats = 10

## Inner maximization

In [3]:
def inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).uniform_(-1, 1).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    for _ in range(num_repeats):
        x.requires_grad_()
        logits = model(x)
        loss = xent(logits, targets)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs - epsilon), inputs + epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes, 
             epsilon=8/255, alpha=2/255, num_repeats=10):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
        logits = model(x)
        
        loss = xent(logits, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/len(dataloader),
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            
            x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
    
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum)

scheduler = [75, 90]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 3.0080 (avg: 0.0079) | acc: 0.0000 (avg: 0.0000) |
Epoch 0 [100/383] | loss: 2.1082 (avg: 0.6022) | acc: 0.1875 (avg: 0.1586) |
Epoch 0 [200/383] | loss: 2.0898 (avg: 1.1583) | acc: 0.2422 (avg: 0.1837) |
Epoch 0 [300/383] | loss: 2.1045 (avg: 1.6998) | acc: 0.2031 (avg: 0.1958) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:18<00:00,  5.04it/s, acc_nat=0.39, acc_rob=0.263]


Epoch 1 [0/383] | loss: 2.0385 (avg: 0.0053) | acc: 0.2266 (avg: 0.2266) |
Epoch 1 [100/383] | loss: 1.9600 (avg: 0.5276) | acc: 0.2734 (avg: 0.2527) |
Epoch 1 [200/383] | loss: 2.0297 (avg: 1.0432) | acc: 0.2109 (avg: 0.2589) |
Epoch 1 [300/383] | loss: 1.9485 (avg: 1.5545) | acc: 0.2031 (avg: 0.2606) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:21<00:00,  4.96it/s, acc_nat=0.446, acc_rob=0.295]


Epoch 2 [0/383] | loss: 1.9274 (avg: 0.0050) | acc: 0.2344 (avg: 0.2344) |
Epoch 2 [100/383] | loss: 2.0152 (avg: 0.5023) | acc: 0.2500 (avg: 0.2922) |
Epoch 2 [200/383] | loss: 1.8145 (avg: 0.9957) | acc: 0.3047 (avg: 0.2918) |
Epoch 2 [300/383] | loss: 1.8529 (avg: 1.4825) | acc: 0.2578 (avg: 0.2943) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:20<00:00,  5.00it/s, acc_nat=0.855, acc_rob=0.456]


Epoch 98 [0/383] | loss: 0.0703 (avg: 0.0002) | acc: 0.9609 (avg: 0.9609) |
Epoch 98 [100/383] | loss: 0.0727 (avg: 0.0274) | acc: 0.9688 (avg: 0.9516) |
Epoch 98 [200/383] | loss: 0.0832 (avg: 0.0557) | acc: 0.9609 (avg: 0.9506) |
Epoch 98 [300/383] | loss: 0.0901 (avg: 0.0825) | acc: 0.9766 (avg: 0.9511) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:20<00:00,  4.98it/s, acc_nat=0.851, acc_rob=0.475]


Epoch 99 [0/383] | loss: 0.1388 (avg: 0.0004) | acc: 0.9453 (avg: 0.9453) |
Epoch 99 [100/383] | loss: 0.0777 (avg: 0.0288) | acc: 0.9609 (avg: 0.9508) |
Epoch 99 [200/383] | loss: 0.1344 (avg: 0.0559) | acc: 0.9375 (avg: 0.9518) |
Epoch 99 [300/383] | loss: 0.0692 (avg: 0.0832) | acc: 0.9844 (avg: 0.9516) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:23<00:00,  4.92it/s, acc_nat=0.852, acc_rob=0.473]
